In [3]:
# validate the verion (3.5 version is more compatible with minio)
# jupyter-pyspark image: quay.io/jupyter/pyspark-notebook:spark-3.5.1
import pyspark 
print("PySpark:", pyspark.__version__) 
print("Spark:", spark.version) 
print("Hadoop:", spark.sparkContext._jsc.hadoopConfiguration().get("hadoop.version"))
print(spark._jsc.hadoopConfiguration().get("fs.s3a.endpoint"))

PySpark: 3.5.1
Spark: 3.5.1
Hadoop: None
http://minio.airflow:9000


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 08:10:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/11 08:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/11 08:10:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
data = [
    ("James", "Smith", "36636", "M", 3000),
    ("Michael", "Rose", "40288", "M", 4000),
    ("Robert", "Williams", "42114", "M", 4000),
    ("Maria", "Jones", "39192", "F", 4000),
    ("Jen", "Brown", "39193", "F", -1)
]

schema = StructType(
    [
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("id", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("salary", IntegerType(), True),
    ]
)

df = spark.createDataFrame(data, schema)

In [5]:
df.show()

+----------+---------+-----+------+------+
|first_name|last_name|   id|gender|salary|
+----------+---------+-----+------+------+
|     James|    Smith|36636|     M|  3000|
|   Michael|     Rose|40288|     M|  4000|
|    Robert| Williams|42114|     M|  4000|
|     Maria|    Jones|39192|     F|  4000|
|       Jen|    Brown|39193|     F|    -1|
+----------+---------+-----+------+------+



In [6]:
# write minio
df.write.mode("overwrite").parquet("s3a://datalake/bronze/jupyter_minio_t1")

In [7]:
# read minio
df = spark.read.format("parquet").load("s3a://datalake/bronze/jupyter_minio_t1")
df.show()

+----------+---------+-----+------+------+
|first_name|last_name|   id|gender|salary|
+----------+---------+-----+------+------+
|    Robert| Williams|42114|     M|  4000|
|   Michael|     Rose|40288|     M|  4000|
|     James|    Smith|36636|     M|  3000|
|     Maria|    Jones|39192|     F|  4000|
|       Jen|    Brown|39193|     F|    -1|
+----------+---------+-----+------+------+

